In [ ]:
# 设置工作路径
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas")
!ls

In [ ]:
# 合并不同时间的同模式文件
# 运行前在路径中新建“tas_mergetime”文件夹
import subprocess

os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_mergetime") # 新建文件夹
# 获取当前目录下所有文件的列表
file_list = os.listdir()
# 去除文件名后续日期
for i in range(len(file_list)):
    file_list[i]=file_list[i][0:-19]
# 去除重复项
file_list=list(set(file_list)) 

for file in file_list:
    # python文件路径变量定义
    file_path = file+'*.nc'
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_mergetime/'+file+'mergetime.nc'
    # 构建cdo命令
    command = f'cdo -mergetime {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_mergetime")
!ls

In [ ]:
# 筛选进行运算的年份(时间)
# 运行前在路径中新建“tas_seltime”文件夹
import subprocess

os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_seltime") # 新建文件夹
# 获取路径文件列表
file_list = os.listdir()
# 筛选模式的数据年份
for file in file_list:
    # 定义路径变量：文件、历史文件、预测文件
    file_path = file
    # 分historical与其它模式分别构建命令
    output_path1 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_seltime/'+file[0:-12]+'197101-200012.nc'
    output_path2 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_seltime/'+file[0:-12]+'200101-201412.nc'
    output_path3 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_seltime/'+file[0:-12]+'201501-206012.nc'
    # 构建cdo命令
    command1 = f'cdo -selyear,1971/2000 {file_path} {output_path1}'
    command2 = f'cdo -selyear,2001/2014 {file_path} {output_path2}'
    command3 = f'cdo -selyear,2015/2060 {file_path} {output_path3}'
    # 分类运行时间筛选的cdo命令
    if "historical" in file:
        subprocess.run(command1, shell=True)
        subprocess.run(command2, shell=True)
    else:
        subprocess.run(command3, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_seltime")
!ls

In [ ]:
!cdo showyear tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_197101-200012.nc

In [ ]:
# 求各模式文件的年均气温
# 运行前在路径中新建“tas_yearmean”文件夹
#os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_seltime")

os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean") # 新建文件夹
file_list = os.listdir()
# 筛选模式的数据年份
for file in file_list:
    # 定义路径变量：文件、历史文件、预测文件
    file_path = file
    output_path1 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean/'+file[0:-16]+'1971-2000_yearmean.nc'
    output_path2 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean/'+file[0:-16]+'2001-2014_yearmean.nc'
    output_path3 = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean/'+file[0:-16]+'2015-2060_yearmean.nc'
    # 构建cdo命令
    command1 = f'cdo -yearmean {file_path} {output_path1}'
    command2 = f'cdo -yearmean {file_path} {output_path2}'
    command3 = f'cdo -yearmean {file_path} {output_path3}'
    # 分类运行时间筛选的cdo命令
    if "197101-200012" in file:
        subprocess.run(command1, shell=True)
    elif "200101-201412" in file:
        subprocess.run(command2, shell=True)
    else:
        subprocess.run(command3, shell=True)

In [ ]:
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean")
!ls

In [ ]:
!cdo info tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearmean.nc

In [ ]:
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000.nc

In [ ]:
!cdo griddes tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearmean.nc

In [ ]:
# 修正格网格式

# 运行前在路径中新建“tas_cngrid”文件夹
import subprocess
import os
# 定义处理路径
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_yearmean")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_cngrid") # 新建文件夹
# 提取格网形式
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000.nc >grid_cn
# 修改格网形式
!sed -i "s/generic/lonlat/g" grid_cn
grid_cn=!cat grid_cn

file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_cngrid/'+file[0:-3]+'_cngrid.nc'
    # 构建cdo命令
    command = f'cdo setgrid,grid_cn {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)
# 修改原观测文件格网形式至统一
!cdo setgrid,grid_cn /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000.nc /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_cngrid")
!ls

In [ ]:
!cdo griddes /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc

In [ ]:
!cdo griddes tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearmean_cngrid.nc

In [ ]:
# 裁剪研究区域

# 运行前在路径中新建“tas_china”文件夹
import os
import subprocess
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_cngrid")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china") # 新建文件夹
# 筛选区域
# 获取文件列表
file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china/'+file[0:-3]+'_china.nc'
    # 构建cdo命令
    command = f'cdo -sellonlatbox,67,137,14,55 {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china")
!ls

In [ ]:
!cdo griddes tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearmean_cngrid_china.nc

In [ ]:
# 降尺度
# 运行前在路径中新建“tas_china_cngridbil”文件夹
# 把grid_cn文件粘贴至工作路径
import os
import subprocess
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china")
#os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china_cngridbil") # 新建文件夹
# 获取文件列表
file_list = os.listdir()

for file in file_list:
    # 定义路径变量：文件、输出文件
    file_path = file
    output_path = '/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china_cngridbil/'+file[0:-15]+'cngridbil_china.nc'
    # 构建cdo命令
    command = f'cdo -remapbil,grid_cn {file_path} {output_path}'
    # 运行cdo命令
    subprocess.run(command, shell=True)

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china_cngridbil")
!ls

In [ ]:
!cdo griddes tas_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_1971-2000_yearmean_cngridbil_china.nc

In [ ]:
# 模式平均

# 运行前在路径中新建“tas_modemean”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_china_cngridbil")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean") # 新建文件夹
!cdo -ensmean  *historical_r1i1p1f1_1971-2000*.nc /media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean/tas_historical_1971-2000_yearmean_cngridbil_china.nc
!cdo -ensmean  *historical_r1i1p1f1_2001-2014*.nc /media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean/tas_historical_2001-2014_yearmean_cngridbil_china.nc
!cdo -ensmean  *ssp126*.nc /media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean/tas_ssp126_2015-2060_yearmean_cngridbil_china.nc
!cdo -ensmean  *ssp245*.nc /media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean/tas_ssp245_2015-2060_yearmean_cngridbil_china.nc
!cdo -ensmean  *ssp585*.nc /media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean/tas_ssp585_2015-2060_yearmean_cngridbil_china.nc

In [ ]:
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
!ls

In [ ]:
!cdo griddes tas_ssp126_2015-2060_yearmean_cngridbil_china.nc

In [ ]:
!cdo -timmean  tas_historical_1971-2000_yearmean_cngridbil_china.nc tas_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
!cdo info tas_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
!cdo info /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc

In [ ]:
!cdo info tas_add_delta.nc

In [ ]:
# 求加性偏差
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
!cdo -s -f nc -sub  -sellevel,0 /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc -sellevel,2 -subc,273.15 tas_historical_1971-2000_timmean_cngridbil_china.nc tas_add_delta.nc

In [ ]:
# 订正gcm

# 运行前在路径中新建“tas_downscaled”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled") # 新建文件夹
!cdo -s -f nc -add  -subc,273.15 tas_historical_2001-2014_yearmean_cngridbil_china.nc tas_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled/tas_historical_2001-2014_yearmean_downscaled_china.nc
!cdo -s -f nc -add  -subc,273.15 tas_ssp126_2015-2060_yearmean_cngridbil_china.nc tas_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled/tas_ssp126_2015-2060_yearmean_downscaled_china.nc
!cdo -s -f nc -add  -subc,273.15 tas_ssp245_2015-2060_yearmean_cngridbil_china.nc tas_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled/tas_ssp245_2015-2060_yearmean_downscaled_china.nc
!cdo -s -f nc -add  -subc,273.15 tas_ssp585_2015-2060_yearmean_cngridbil_china.nc tas_add_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled/tas_ssp585_2015-2060_yearmean_downscaled_china.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_add_downscaled")
!ls

In [ ]:
!cdo info tas_historical_2001-2014_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp126_2015-2060_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp245_2015-2060_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp585_2015-2060_yearmean_downscaled_china.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
!cdo info tas_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
!cdo info /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc

In [ ]:
! cdo info -div  -sellevel,0 /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc -sellevel,2 -subc,273.15 tas_historical_1971-2000_timmean_cngridbil_china.nc

In [ ]:
# 求比例偏差
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
!cdo -s -f nc -div /media/sf_VM_VirtualBox_Ubnutu/obs_tas_1971-2000_cngrid.nc -subc,273.15 tas_historical_1971-2000_timmean_cngridbil_china.nc tas_mul_delta.nc

In [ ]:
!cdo info tas_mul_delta.nc

In [ ]:
# 订正gcm

# 运行前在路径中新建“tas_downscaled”文件夹
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_111/tas_modemean")
#os.mkdir("/media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled") # 新建文件夹
!cdo -s -f nc -mul  -subc,273.15 tas_historical_2001-2014_yearmean_cngridbil_china.nc tas_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled/tas_historical_2001-2014_yearmean_downscaled_china.nc
!cdo -s -f nc -mul  -subc,273.15 tas_ssp126_2015-2060_yearmean_cngridbil_china.nc tas_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled/tas_ssp126_2015-2060_yearmean_downscaled_china.nc
!cdo -s -f nc -mul  -subc,273.15 tas_ssp245_2015-2060_yearmean_cngridbil_china.nc tas_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled/tas_ssp245_2015-2060_yearmean_downscaled_china.nc
!cdo -s -f nc -mul  -subc,273.15 tas_ssp585_2015-2060_yearmean_cngridbil_china.nc tas_mul_delta.nc /media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled/tas_ssp585_2015-2060_yearmean_downscaled_china.nc

In [ ]:
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled")
!ls

In [ ]:
!cdo info tas_historical_2001-2014_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp126_2015-2060_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp245_2015-2060_yearmean_downscaled_china.nc

In [ ]:
!cdo info tas_ssp585_2015-2060_yearmean_downscaled_china.nc

In [ ]:
import os
os.chdir("/media/sf_VM_VirtualBox_Ubnutu/tas_mul_downscaled")
!ls

In [ ]:
!cdo timmean tas_historical_2001-2014_yearmean_downscaled_china.nc tas_historical_2001-2014_timmean_downscaled_china.nc